In [3]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np

# 初始化H2O集群
h2o.init(port=54322)
 
# h2o导入方式
# train_1 = h2o.import_file("/kaggle/input/titanic/train.csv")
# test_1 = h2o.import_file("/kaggle/input/titanic/test.csv")
 
# pandas导入方式
train = pd.read_excel('副本催化剂spssau3 - 副本.xlsx')

Checking whether there is an H2O instance running at http://localhost:54322. connected.


H2O_cluster_uptime:,1 min 14 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,2 months and 9 days
H2O_cluster_name:,H2O_from_python_wangs_wboiot
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.979 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [4]:
train

,Catalyst Loading Rate(%),Reaction \nTime(min),Temperature(K),Catalyst Dosage(g),NaBH4(mol/L),NaOH(mol/L),Hydrogen \nProduction\n(ml/(min·g))
0,63.58,1,303,0.01,1.0,0.25,62.650610
1,63.58,2,303,0.01,1.0,0.25,180.428600
2,63.58,3,303,0.01,1.0,0.25,340.204950
3,63.58,4,303,0.01,1.0,0.25,522.349980
4,63.58,5,303,0.01,1.0,0.25,721.842160
...,...,...,...,...,...,...,...
633,63.58,11,333,0.01,5.0,3.00,710.022800
634,63.58,11,333,0.01,5.0,4.00,774.149297
635,63.58,11,343,0.01,5.0,4.00,781.844400
636,63.58,11,353,0.01,5.0,4.00,756.968156


## 9:1

In [5]:
X = train.iloc[:,:-1]
y = train.iloc[:,-1]
# 将数据分割为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 将数据转换为Pandas数据框
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)
# 将Pandas数据框转换为H2O数据框
train_h2o = h2o.H2OFrame(train_df)
test_h2o = h2o.H2OFrame(test_df)

# 定义特征和目标变量
x = train_h2o.drop('Hydrogen \nProduction\n(ml/(min·g))').columns
y = 'Hydrogen \nProduction\n(ml/(min·g))'

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


## 60 S

In [6]:
aml = H2OAutoML(max_runtime_secs=60,  # 训练的时间预算（秒）
                stopping_metric='rmse',  # 评估指标，采用平均绝对误差
                seed=42)

# 训练模型
aml.train(x=x, y=y, training_frame=train_h2o)

AutoML progress: |█
16:18:31.746: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),2/5
# GBM base models (used / total),1/1
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),1/1
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [7]:
lb = aml.leaderboard
print(lb)

model_id                                                    rmse      mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_BestOfFamily_4_AutoML_1_20241107_161831  253.324  64173    170.155  nan                          64173
GBM_grid_1_AutoML_1_20241107_161831_model_55             253.842  64435.7  174.398  nan                          64435.7
GBM_grid_1_AutoML_1_20241107_161831_model_51             256.639  65863.6  173.698    0.352434                   65863.6
GBM_grid_1_AutoML_1_20241107_161831_model_1              262.317  68810.1  178.617    0.349774                   68810.1
GBM_grid_1_AutoML_1_20241107_161831_model_122            265.699  70596    184.413    0.359927                   70596
StackedEnsemble_AllModels_3_AutoML_1_20241107_161831     266.799  71181.5  198.479    0.397828                   71181.5
StackedEnsemble_AllModels_4_AutoML_1_20241107_161831     266.855  71211.3  198.053    0.399592                   71211.3
StackedEnsemble_BestOfFamily_5_AutoM

In [8]:
# 获取模型 ID 列表
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])

# 遍历模型，查看性能
for model_id in model_ids:
    model = h2o.get_model(model_id)
    perf = model.model_performance(test_data=test_h2o)
    print(f"Model ID: {model_id}")
    print(perf)

C:\Users\wangs\.conda\envs\H2O\lib\site-packages\h2o\frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Model ID: StackedEnsemble_BestOfFamily_4_AutoML_1_20241107_161831
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 58923.61769386169
RMSE: 242.74187461964962
MAE: 163.05359104682933
RMSLE: NaN
Mean Residual Deviance: 58923.61769386169
R^2: 0.8147343664092683
Null degrees of freedom: 63
Residual degrees of freedom: 61
Null deviance: 20398359.936008632
Residual deviance: 3771111.5324071483
AIC: 892.5999574528313
Model ID: GBM_grid_1_AutoML_1_20241107_161831_model_55
ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 58546.0208073971
RMSE: 241.96285005636113
MAE: 171.24452139512846
RMSLE: 0.3359238135151634
Mean Residual Deviance: 58546.0208073971
Model ID: GBM_grid_1_AutoML_1_20241107_161831_model_51
ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 58558.0600716436
RMSE: 241.98772710954495
MAE: 170.80871273331064
RMSLE: 0.3268579487377155
Mean Residual Deviance: 58558.0600716436
Model ID: GBM_grid_1_AutoML_1_20241107_161831_model_1
M

In [9]:
# 获取最佳模型
best_model = aml.leader
perf = best_model.model_performance(test_h2o)
print(perf)

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 58923.61769386169
RMSE: 242.74187461964962
MAE: 163.05359104682933
RMSLE: NaN
Mean Residual Deviance: 58923.61769386169
R^2: 0.8147343664092683
Null degrees of freedom: 63
Residual degrees of freedom: 61
Null deviance: 20398359.936008632
Residual deviance: 3771111.5324071483
AIC: 892.5999574528313


In [10]:
# 打印最佳模型的信息
print(f'Best Model: {best_model}')

Best Model: Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_BestOfFamily_4_AutoML_1_20241107_161831


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       2/5
# GBM base models (used / total)           1/1
# DRF base models (used / total)           0/2
# DeepLearning base models (used / total)  1/1
# GLM base models (used / total)           0/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         None

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 45560.11877287756
RMSE: 213.44816413564573
MAE: 130.90547057252076
RMSLE: NaN
Mean Residual Deviance: 45560.11877287756
R^2

## 300s

In [11]:
aml = H2OAutoML(max_runtime_secs=300,  # 训练的时间预算（秒）
                stopping_metric='rmse',  # 评估指标，采用平均绝对误差
                seed=42)

# 训练模型
aml.train(x=x, y=y, training_frame=train_h2o)

AutoML progress: |█
16:31:13.327: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/211
# GBM base models (used / total),3/187
# DeepLearning base models (used / total),5/21
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [12]:
lb = aml.leaderboard
print(lb)

model_id                                                    rmse      mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_6_AutoML_2_20241107_163113     246.734  60877.6  160.669  nan                          60877.6
StackedEnsemble_BestOfFamily_4_AutoML_2_20241107_163113  249.629  62314.9  165.894  nan                          62314.9
StackedEnsemble_BestOfFamily_6_AutoML_2_20241107_163113  249.907  62453.5  166.274  nan                          62453.5
StackedEnsemble_AllModels_5_AutoML_2_20241107_163113     252.541  63777.1  160.328    0.35451                    63777.1
GBM_grid_1_AutoML_2_20241107_163113_model_55             253.842  64435.7  174.398  nan                          64435.7
GBM_grid_1_AutoML_2_20241107_163113_model_51             256.639  65863.6  173.698    0.352434                   65863.6
StackedEnsemble_Best1000_1_AutoML_2_20241107_163113      257.498  66305.1  188.024    0.383993                   66305.1
StackedEnsemble_AllModels_4_Auto

In [13]:
# 获取模型 ID 列表
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])

# 遍历模型，查看性能
for model_id in model_ids:
    model = h2o.get_model(model_id)
    perf = model.model_performance(test_data=test_h2o)
    print(f"Model ID: {model_id}")
    print(perf)

C:\Users\wangs\.conda\envs\H2O\lib\site-packages\h2o\frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Model ID: StackedEnsemble_AllModels_6_AutoML_2_20241107_163113
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 53151.981346934284
RMSE: 230.54713476192734
MAE: 152.9972216119374
RMSLE: 0.32631960924081144
Mean Residual Deviance: 53151.981346934284
R^2: 0.8328813490033156
Null degrees of freedom: 63
Residual degrees of freedom: 55
Null deviance: 20398359.936008632
Residual deviance: 3401726.806203794
AIC: 898.0024145845846
Model ID: StackedEnsemble_BestOfFamily_4_AutoML_2_20241107_163113
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 54996.35702421107
RMSE: 234.51302101207744
MAE: 156.17107354390015
RMSLE: NaN
Mean Residual Deviance: 54996.35702421107
R^2: 0.8270823257626632
Null degrees of freedom: 63
Residual degrees of freedom: 61
Null deviance: 20398359.936008632
Residual deviance: 3519766.8495495087
AIC: 888.1855547204367
Model ID: StackedEnsemble_BestOfFamily_6_AutoML_2_20241107_163113
ModelMetricsRegressionGLM: stackede

In [14]:
# 获取最佳模型
best_model = aml.leader
perf = best_model.model_performance(test_h2o)
print(perf)

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 53151.981346934284
RMSE: 230.54713476192734
MAE: 152.9972216119374
RMSLE: 0.32631960924081144
Mean Residual Deviance: 53151.981346934284
R^2: 0.8328813490033156
Null degrees of freedom: 63
Residual degrees of freedom: 55
Null deviance: 20398359.936008632
Residual deviance: 3401726.806203794
AIC: 898.0024145845846


In [15]:
# 打印最佳模型的信息
print(f'Best Model: {best_model}')

Best Model: Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_6_AutoML_2_20241107_163113


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       8/211
# GBM base models (used / total)           3/187
# DeepLearning base models (used / total)  5/21
# DRF base models (used / total)           0/2
# GLM base models (used / total)           0/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         None

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 42405.54870565605
RMSE: 205.92607582736105
MAE: 124.4123007750018
RMSLE: NaN
Mean Residual Deviance: 42405.54870565605
R^

## 900 s

In [16]:
aml = H2OAutoML(max_runtime_secs=900,  # 训练的时间预算（秒）
                stopping_metric='rmse',  # 评估指标，采用平均绝对误差
                seed=42)

# 训练模型
aml.train(x=x, y=y, training_frame=train_h2o)

AutoML progress: |█
16:53:38.522: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),10/227
# GBM base models (used / total),3/187
# DeepLearning base models (used / total),7/37
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [17]:
lb = aml.leaderboard
print(lb)

model_id                                                    rmse      mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_6_AutoML_3_20241107_165338     242.341  58729.1  159.918  nan                          58729.1
StackedEnsemble_BestOfFamily_6_AutoML_3_20241107_165338  247.036  61026.8  163.397  nan                          61026.8
StackedEnsemble_BestOfFamily_4_AutoML_3_20241107_165338  247.151  61083.8  163.418  nan                          61083.8
StackedEnsemble_AllModels_4_AutoML_3_20241107_165338     252.266  63637.9  183.658    0.375446                   63637.9
StackedEnsemble_Best1000_1_AutoML_3_20241107_165338      252.626  63819.7  183.84     0.373897                   63819.7
GBM_grid_1_AutoML_3_20241107_165338_model_55             253.842  64435.7  174.398  nan                          64435.7
StackedEnsemble_AllModels_5_AutoML_3_20241107_165338     253.987  64509.6  162.291    0.350634                   64509.6
DeepLearning_grid_1_AutoML_3_202

In [18]:
# 获取模型 ID 列表
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])

# 遍历模型，查看性能
for model_id in model_ids:
    model = h2o.get_model(model_id)
    perf = model.model_performance(test_data=test_h2o)
    print(f"Model ID: {model_id}")
    print(perf)

C:\Users\wangs\.conda\envs\H2O\lib\site-packages\h2o\frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Model ID: StackedEnsemble_AllModels_6_AutoML_3_20241107_165338
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 51908.111365680656
RMSE: 227.83351677415826
MAE: 151.55453958275075
RMSLE: 0.30913681339722127
Mean Residual Deviance: 51908.111365680656
R^2: 0.8367922826696551
Null degrees of freedom: 63
Residual degrees of freedom: 53
Null deviance: 20398359.936008632
Residual deviance: 3322119.127403562
AIC: 900.4868743492899
Model ID: StackedEnsemble_BestOfFamily_6_AutoML_3_20241107_165338
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 54610.97473245737
RMSE: 233.68991149054204
MAE: 149.72176180776967
RMSLE: NaN
Mean Residual Deviance: 54610.97473245737
R^2: 0.8282940316498908
Null degrees of freedom: 63
Residual degrees of freedom: 61
Null deviance: 20398359.936008632
Residual deviance: 3495102.3828772716
AIC: 887.7355014636813
Model ID: StackedEnsemble_BestOfFamily_4_AutoML_3_20241107_165338
ModelMetricsRegressionGLM: stacked

In [19]:
# 获取最佳模型
best_model = aml.leader
perf = best_model.model_performance(test_h2o)
print(perf)

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 51908.111365680656
RMSE: 227.83351677415826
MAE: 151.55453958275075
RMSLE: 0.30913681339722127
Mean Residual Deviance: 51908.111365680656
R^2: 0.8367922826696551
Null degrees of freedom: 63
Residual degrees of freedom: 53
Null deviance: 20398359.936008632
Residual deviance: 3322119.127403562
AIC: 900.4868743492899


In [20]:
# 打印最佳模型的信息
print(f'Best Model: {best_model}')

Best Model: Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_6_AutoML_3_20241107_165338


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       10/227
# GBM base models (used / total)           3/187
# DeepLearning base models (used / total)  7/37
# DRF base models (used / total)           0/2
# GLM base models (used / total)           0/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         None

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 42527.91019851435
RMSE: 206.22296234540505
MAE: 124.94583374279222
RMSLE: 0.3022936557903484
Mean Residual Deviance: 425

## 1200 s

In [21]:
aml = H2OAutoML(max_runtime_secs=1200,  # 训练的时间预算（秒）
                stopping_metric='rmse',  # 评估指标，采用平均绝对误差
                seed=42)

# 训练模型
aml.train(x=x, y=y, training_frame=train_h2o)

AutoML progress: |█
17:29:08.899: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),9/235
# GBM base models (used / total),2/187
# DeepLearning base models (used / total),7/45
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [22]:
lb = aml.leaderboard
print(lb)

model_id                                                    rmse      mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_6_AutoML_4_20241107_172908     235.673  55541.9  146.074  nan                          55541.9
StackedEnsemble_BestOfFamily_4_AutoML_4_20241107_172908  243.499  59291.6  160.567  nan                          59291.6
StackedEnsemble_BestOfFamily_5_AutoML_4_20241107_172908  243.875  59475.1  158.356    0.369117                   59475.1
StackedEnsemble_BestOfFamily_7_AutoML_4_20241107_172908  244.082  59576.1  159.321  nan                          59576.1
StackedEnsemble_Best1000_1_AutoML_4_20241107_172908      245.218  60131.9  174.488    0.379538                   60131.9
StackedEnsemble_AllModels_4_AutoML_4_20241107_172908     246.889  60954.2  175.582    0.380708                   60954.2
StackedEnsemble_BestOfFamily_6_AutoML_4_20241107_172908  247.184  61099.9  166.058    0.341719                   61099.9
DeepLearning_grid_2_AutoML_4_202

In [23]:
# 获取模型 ID 列表
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])

# 遍历模型，查看性能
for model_id in model_ids:
    model = h2o.get_model(model_id)
    perf = model.model_performance(test_data=test_h2o)
    print(f"Model ID: {model_id}")
    print(perf)

C:\Users\wangs\.conda\envs\H2O\lib\site-packages\h2o\frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Model ID: StackedEnsemble_AllModels_6_AutoML_4_20241107_172908
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 53012.52433226697
RMSE: 230.24448816913505
MAE: 146.3710755218067
RMSLE: 0.3533166752001646
Mean Residual Deviance: 53012.52433226697
R^2: 0.8333198250031302
Null degrees of freedom: 63
Residual degrees of freedom: 54
Null deviance: 20398359.936008632
Residual deviance: 3392801.557265086
AIC: 899.8342745077255
Model ID: StackedEnsemble_BestOfFamily_4_AutoML_4_20241107_172908
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 51355.42357378874
RMSE: 226.61735055769392
MAE: 150.3435145194322
RMSLE: NaN
Mean Residual Deviance: 51355.42357378874
R^2: 0.838530024817035
Null degrees of freedom: 63
Residual degrees of freedom: 61
Null deviance: 20398359.936008632
Residual deviance: 3286747.1087224795
AIC: 883.8017853396095
Model ID: StackedEnsemble_BestOfFamily_5_AutoML_4_20241107_172908
ModelMetricsRegressionGLM: stackedensemb

In [24]:
# 获取最佳模型
best_model = aml.leader
perf = best_model.model_performance(test_h2o)
print(perf)

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 53012.52433226697
RMSE: 230.24448816913505
MAE: 146.3710755218067
RMSLE: 0.3533166752001646
Mean Residual Deviance: 53012.52433226697
R^2: 0.8333198250031302
Null degrees of freedom: 63
Residual degrees of freedom: 54
Null deviance: 20398359.936008632
Residual deviance: 3392801.557265086
AIC: 899.8342745077255


In [25]:
# 打印最佳模型的信息
print(f'Best Model: {best_model}')

Best Model: Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_6_AutoML_4_20241107_172908


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       9/235
# GBM base models (used / total)           2/187
# DeepLearning base models (used / total)  7/45
# DRF base models (used / total)           0/2
# GLM base models (used / total)           0/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         None

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 39466.109081714974
RMSE: 198.6607889889572
MAE: 112.60804380989177
RMSLE: 0.2870365099352893
Mean Residual Deviance: 3946

## 1500 s

In [26]:
aml = H2OAutoML(max_runtime_secs=1500,  # 训练的时间预算（秒）
                stopping_metric='rmse',  # 评估指标，采用平均绝对误差
                seed=42)

# 训练模型
aml.train(x=x, y=y, training_frame=train_h2o)

AutoML progress: |█
18:07:55.689: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/243
# GBM base models (used / total),1/187
# DeepLearning base models (used / total),7/53
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [27]:
lb = aml.leaderboard
print(lb)

model_id                                                    rmse      mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_6_AutoML_5_20241107_180755     233.212  54387.9  147.874  nan                          54387.9
StackedEnsemble_BestOfFamily_5_AutoML_5_20241107_180755  239.226  57229.2  154.749  nan                          57229.2
StackedEnsemble_BestOfFamily_7_AutoML_5_20241107_180755  239.333  57280.4  154.889  nan                          57280.4
DeepLearning_grid_2_AutoML_5_20241107_180755_model_2     241.827  58480.3  162.722    0.356356                   58480.3
StackedEnsemble_Best1000_1_AutoML_5_20241107_180755      243.81   59443.3  173.703    0.375822                   59443.3
StackedEnsemble_BestOfFamily_4_AutoML_5_20241107_180755  244.18   59624    159.445  nan                          59624
StackedEnsemble_AllModels_4_AutoML_5_20241107_180755     245.355  60198.9  174.707    0.378556                   60198.9
DeepLearning_grid_3_AutoML_5_20241

In [28]:
# 获取模型 ID 列表
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])

# 遍历模型，查看性能
for model_id in model_ids:
    model = h2o.get_model(model_id)
    perf = model.model_performance(test_data=test_h2o)
    print(f"Model ID: {model_id}")
    print(perf)

C:\Users\wangs\.conda\envs\H2O\lib\site-packages\h2o\frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Model ID: StackedEnsemble_AllModels_6_AutoML_5_20241107_180755
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 52638.77226808503
RMSE: 229.43141081396206
MAE: 149.05864887458188
RMSLE: 0.30550535104187415
Mean Residual Deviance: 52638.77226808503
R^2: 0.8344949635246016
Null degrees of freedom: 63
Residual degrees of freedom: 55
Null deviance: 20398359.936008632
Residual deviance: 3368881.4251574418
AIC: 897.3814597735602
Model ID: StackedEnsemble_BestOfFamily_5_AutoML_5_20241107_180755
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 47562.454524649176
RMSE: 218.08818061657806
MAE: 142.30216512685234
RMSLE: 0.29132164848126135
Mean Residual Deviance: 47562.454524649176
R^2: 0.8504557490271436
Null degrees of freedom: 63
Residual degrees of freedom: 61
Null deviance: 20398359.936008632
Residual deviance: 3043997.0895775473
AIC: 878.8912655932911
Model ID: StackedEnsemble_BestOfFamily_7_AutoML_5_20241107_180755
ModelMetricsRegre

In [29]:
# 获取最佳模型
best_model = aml.leader
perf = best_model.model_performance(test_h2o)
print(perf)

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 52638.77226808503
RMSE: 229.43141081396206
MAE: 149.05864887458188
RMSLE: 0.30550535104187415
Mean Residual Deviance: 52638.77226808503
R^2: 0.8344949635246016
Null degrees of freedom: 63
Residual degrees of freedom: 55
Null deviance: 20398359.936008632
Residual deviance: 3368881.4251574418
AIC: 897.3814597735602


In [30]:
# 打印最佳模型的信息
print(f'Best Model: {best_model}')

Best Model: Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_6_AutoML_5_20241107_180755


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       8/243
# GBM base models (used / total)           1/187
# DeepLearning base models (used / total)  7/53
# DRF base models (used / total)           0/2
# GLM base models (used / total)           0/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         None

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 41291.2099153163
RMSE: 203.20238658863312
MAE: 120.17779809242991
RMSLE: 0.2861405005073726
Mean Residual Deviance: 41291

## 1800 s

In [31]:
aml = H2OAutoML(max_runtime_secs=1800,  # 训练的时间预算（秒）
                stopping_metric='rmse',  # 评估指标，采用平均绝对误差
                seed=42)

# 训练模型
aml.train(x=x, y=y, training_frame=train_h2o)

AutoML progress: |█
18:52:06.61: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),11/249
# GBM base models (used / total),1/187
# DeepLearning base models (used / total),10/59
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [32]:
lb = aml.leaderboard
print(lb)

model_id                                                    rmse      mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_6_AutoML_6_20241107_185206     235.76   55582.6  147.601    0.335665                   55582.6
StackedEnsemble_BestOfFamily_5_AutoML_6_20241107_185206  240.922  58043.3  157.469  nan                          58043.3
StackedEnsemble_BestOfFamily_7_AutoML_6_20241107_185206  241.527  58335.4  158.534    0.360997                   58335.4
DeepLearning_grid_2_AutoML_6_20241107_185206_model_2     243.061  59078.6  165.177    0.35513                    59078.6
StackedEnsemble_BestOfFamily_4_AutoML_6_20241107_185206  244.633  59845.5  159.051  nan                          59845.5
DeepLearning_grid_3_AutoML_6_20241107_185206_model_2     245.623  60330.8  166.318    0.350678                   60330.8
StackedEnsemble_BestOfFamily_6_AutoML_6_20241107_185206  246.803  60912    165.65     0.346539                   60912
StackedEnsemble_Best1000_1_AutoML_

In [34]:
# 获取模型 ID 列表
model_ids = list(lb['model_id'].as_data_frame().iloc[:20,0])

# 遍历模型，查看性能
for model_id in model_ids:
    model = h2o.get_model(model_id)
    perf = model.model_performance(test_data=test_h2o)
    print(f"Model ID: {model_id}")
    print(perf)

Model ID: StackedEnsemble_AllModels_6_AutoML_6_20241107_185206
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 51250.327619729054
RMSE: 226.38535204321204
MAE: 149.00150363949072
RMSLE: 0.33032120593874026
Mean Residual Deviance: 51250.327619729054
R^2: 0.8388604639393891
Null degrees of freedom: 63
Residual degrees of freedom: 52
Null deviance: 20398359.936008632
Residual deviance: 3280020.9676626595
AIC: 901.6706787820574
Model ID: StackedEnsemble_BestOfFamily_5_AutoML_6_20241107_185206
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 51709.900884752526
RMSE: 227.3981109964472
MAE: 156.23196128057916
RMSLE: 0.3553575022826043
Mean Residual Deviance: 51709.900884752526
R^2: 0.837415489318715
Null degrees of freedom: 63
Residual degrees of freedom: 61
Null deviance: 20398359.936008632
Residual deviance: 3309433.6566241616
AIC: 884.2420233629277
Model ID: StackedEnsemble_BestOfFamily_7_AutoML_6_20241107_185206
ModelMetricsRegres

In [35]:
# 获取最佳模型
best_model = aml.leader
perf = best_model.model_performance(test_h2o)
print(perf)

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 51250.327619729054
RMSE: 226.38535204321204
MAE: 149.00150363949072
RMSLE: 0.33032120593874026
Mean Residual Deviance: 51250.327619729054
R^2: 0.8388604639393891
Null degrees of freedom: 63
Residual degrees of freedom: 52
Null deviance: 20398359.936008632
Residual deviance: 3280020.9676626595
AIC: 901.6706787820574


In [36]:
# 打印最佳模型的信息
print(f'Best Model: {best_model}')

Best Model: Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_6_AutoML_6_20241107_185206


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       11/249
# GBM base models (used / total)           1/187
# DeepLearning base models (used / total)  10/59
# DRF base models (used / total)           0/2
# GLM base models (used / total)           0/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         None

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 40743.65799484956
RMSE: 201.8505833403747
MAE: 117.26298270053837
RMSLE: 0.2937077978015787
Mean Residual Deviance: 407

## 2100 s

In [37]:
aml = H2OAutoML(max_runtime_secs=2100,  # 训练的时间预算（秒）
                stopping_metric='rmse',  # 评估指标，采用平均绝对误差
                seed=42)

# 训练模型
aml.train(x=x, y=y, training_frame=train_h2o)

AutoML progress: |█
19:27:47.639: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),10/258
# GBM base models (used / total),2/187
# DeepLearning base models (used / total),8/68
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [38]:
lb = aml.leaderboard
print(lb)

model_id                                                    rmse      mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_6_AutoML_7_20241107_192747     237.566  56437.4  149.986  nan                          56437.4
StackedEnsemble_BestOfFamily_5_AutoML_7_20241107_192747  240.807  57988.2  154.825  nan                          57988.2
StackedEnsemble_BestOfFamily_7_AutoML_7_20241107_192747  241.213  58183.7  155.414    0.365039                   58183.7
StackedEnsemble_BestOfFamily_6_AutoML_7_20241107_192747  242.793  58948.5  162.301    0.34857                    58948.5
DeepLearning_grid_2_AutoML_7_20241107_192747_model_2     243.437  59261.4  162.294    0.361279                   59261.4
StackedEnsemble_BestOfFamily_4_AutoML_7_20241107_192747  244.237  59652    160.998  nan                          59652
StackedEnsemble_Best1000_1_AutoML_7_20241107_192747      248.725  61864.1  179.347    0.383402                   61864.1
StackedEnsemble_AllModels_4_AutoML

In [39]:
# 获取模型 ID 列表
model_ids = list(lb['model_id'].as_data_frame().iloc[:20,0])

# 遍历模型，查看性能
for model_id in model_ids:
    model = h2o.get_model(model_id)
    perf = model.model_performance(test_data=test_h2o)
    print(f"Model ID: {model_id}")
    print(perf)

C:\Users\wangs\.conda\envs\H2O\lib\site-packages\h2o\frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Model ID: StackedEnsemble_AllModels_6_AutoML_7_20241107_192747
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 50526.19803103691
RMSE: 224.78033283861137
MAE: 151.4254607719879
RMSLE: 0.3488144455054204
Mean Residual Deviance: 50526.19803103691
R^2: 0.8411372475501283
Null degrees of freedom: 63
Residual degrees of freedom: 53
Null deviance: 20398359.936008632
Residual deviance: 3233676.6739863623
AIC: 898.7599564831028
Model ID: StackedEnsemble_BestOfFamily_5_AutoML_7_20241107_192747
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 48323.679600888325
RMSE: 219.8264761144306
MAE: 149.6051272616419
RMSLE: 0.3358550888825193
Mean Residual Deviance: 48323.679600888325
R^2: 0.8480623310468134
Null degrees of freedom: 63
Residual degrees of freedom: 61
Null deviance: 20398359.936008632
Residual deviance: 3092715.494456853
AIC: 879.9074589951591
Model ID: StackedEnsemble_BestOfFamily_7_AutoML_7_20241107_192747
ModelMetricsRegressionG

In [40]:
# 获取最佳模型
best_model = aml.leader
perf = best_model.model_performance(test_h2o)
print(perf)

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 50526.19803103691
RMSE: 224.78033283861137
MAE: 151.4254607719879
RMSLE: 0.3488144455054204
Mean Residual Deviance: 50526.19803103691
R^2: 0.8411372475501283
Null degrees of freedom: 63
Residual degrees of freedom: 53
Null deviance: 20398359.936008632
Residual deviance: 3233676.6739863623
AIC: 898.7599564831028


In [41]:
# 打印最佳模型的信息
print(f'Best Model: {best_model}')

Best Model: Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_6_AutoML_7_20241107_192747


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       10/258
# GBM base models (used / total)           2/187
# DeepLearning base models (used / total)  8/68
# DRF base models (used / total)           0/2
# GLM base models (used / total)           0/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         None

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 40012.15777620344
RMSE: 200.0303921313045
MAE: 118.86096625309638
RMSLE: 0.3037518411858915
Mean Residual Deviance: 4001

## 2400s

In [42]:
aml = H2OAutoML(max_runtime_secs=2400,  # 训练的时间预算（秒）
                stopping_metric='rmse',  # 评估指标，采用平均绝对误差
                seed=42)

# 训练模型
aml.train(x=x, y=y, training_frame=train_h2o)

AutoML progress: |█
20:08:45.957: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),10/266
# GBM base models (used / total),1/187
# DeepLearning base models (used / total),9/76
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [43]:
lb = aml.leaderboard
print(lb)

model_id                                                    rmse      mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_6_AutoML_8_20241107_200845     237.118  56224.8  148.994    0.323562                   56224.8
StackedEnsemble_BestOfFamily_4_AutoML_8_20241107_200845  243.41   59248.3  161.788  nan                          59248.3
StackedEnsemble_BestOfFamily_5_AutoML_8_20241107_200845  243.428  59257.3  160.554    0.33855                    59257.3
StackedEnsemble_BestOfFamily_7_AutoML_8_20241107_200845  243.81   59443.4  161.168    0.333742                   59443.4
StackedEnsemble_Best1000_1_AutoML_8_20241107_200845      247.562  61287.1  177.779    0.381734                   61287.1
StackedEnsemble_AllModels_4_AutoML_8_20241107_200845     248.851  61926.6  178.779    0.384136                   61926.6
DeepLearning_grid_3_AutoML_8_20241107_200845_model_2     250.261  62630.4  173.369    0.35068                    62630.4
StackedEnsemble_AllModels_3_Auto

In [44]:
# 获取模型 ID 列表
model_ids = list(lb['model_id'].as_data_frame().iloc[:20,0])

# 遍历模型，查看性能
for model_id in model_ids:
    model = h2o.get_model(model_id)
    perf = model.model_performance(test_data=test_h2o)
    print(f"Model ID: {model_id}")
    print(perf)

C:\Users\wangs\.conda\envs\H2O\lib\site-packages\h2o\frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Model ID: StackedEnsemble_AllModels_6_AutoML_8_20241107_200845
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 45900.07403672488
RMSE: 214.24302564313473
MAE: 136.48277967726074
RMSLE: 0.36930030965258687
Mean Residual Deviance: 45900.07403672488
R^2: 0.8556825491867043
Null degrees of freedom: 63
Residual degrees of freedom: 53
Null deviance: 20398359.936008632
Residual deviance: 2937604.7383503923
AIC: 892.6143408292622
Model ID: StackedEnsemble_BestOfFamily_4_AutoML_8_20241107_200845
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 52084.382730831865
RMSE: 228.2200313969654
MAE: 153.3039145357041
RMSLE: NaN
Mean Residual Deviance: 52084.382730831865
R^2: 0.8362380562418361
Null degrees of freedom: 63
Residual degrees of freedom: 61
Null deviance: 20398359.936008632
Residual deviance: 3333400.4947732394
AIC: 884.7038395884082
Model ID: StackedEnsemble_BestOfFamily_5_AutoML_8_20241107_200845
ModelMetricsRegressionGLM: stackede

In [47]:
model_ids

['StackedEnsemble_AllModels_6_AutoML_8_20241107_200845',
 'StackedEnsemble_BestOfFamily_4_AutoML_8_20241107_200845',
 'StackedEnsemble_BestOfFamily_5_AutoML_8_20241107_200845',
 'StackedEnsemble_BestOfFamily_7_AutoML_8_20241107_200845',
 'StackedEnsemble_Best1000_1_AutoML_8_20241107_200845',
 'StackedEnsemble_AllModels_4_AutoML_8_20241107_200845',
 'DeepLearning_grid_3_AutoML_8_20241107_200845_model_2',
 'StackedEnsemble_AllModels_3_AutoML_8_20241107_200845',
 'StackedEnsemble_BestOfFamily_6_AutoML_8_20241107_200845',
 'DeepLearning_grid_2_AutoML_8_20241107_200845_model_2',
 'StackedEnsemble_AllModels_5_AutoML_8_20241107_200845',
 'GBM_grid_1_AutoML_8_20241107_200845_model_55',
 'DeepLearning_grid_1_AutoML_8_20241107_200845_model_4',
 'DeepLearning_grid_1_AutoML_8_20241107_200845_model_11',
 'GBM_grid_1_AutoML_8_20241107_200845_model_51',
 'DeepLearning_grid_1_AutoML_8_20241107_200845_model_21',
 'DeepLearning_grid_2_AutoML_8_20241107_200845_model_4',
 'DeepLearning_grid_2_AutoML_8_202

In [45]:
# 获取最佳模型
best_model = aml.leader
perf = best_model.model_performance(test_h2o)
print(perf)

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 45900.07403672488
RMSE: 214.24302564313473
MAE: 136.48277967726074
RMSLE: 0.36930030965258687
Mean Residual Deviance: 45900.07403672488
R^2: 0.8556825491867043
Null degrees of freedom: 63
Residual degrees of freedom: 53
Null deviance: 20398359.936008632
Residual deviance: 2937604.7383503923
AIC: 892.6143408292622


In [46]:
# 打印最佳模型的信息
print(f'Best Model: {best_model}')

Best Model: Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_6_AutoML_8_20241107_200845


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       10/266
# GBM base models (used / total)           1/187
# DeepLearning base models (used / total)  9/76
# DRF base models (used / total)           0/2
# GLM base models (used / total)           0/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         None

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 39617.892636236524
RMSE: 199.0424392842806
MAE: 112.11698929253275
RMSLE: 0.2732665483618557
Mean Residual Deviance: 396

In [ ]:
ra_plot = best_model.residual_analysis_plot(test_h2o)
plt.title('Residual')
plt.savefig('ra_plot.png',dpi =600)

In [ ]:
# 指定特征和目标列
x = data.columns
y = "CAPSULE"  # 假设是二分类任务，CAPSULE是目标列
x.remove(y)

# 分割数据集为训练集和测试集
train, test = data.split_frame(ratios=[0.8], seed=1)

# 运行 H2O AutoML
aml = H2OAutoML(max_runtime_secs=120, seed=1)
aml.train(x=x, y=y, training_frame=train)

# 获取最佳模型
best_model = aml.leader

In [ ]:
# 选择预测变量和目标
hf['Hydrogen Production'] = hf['Hydrogen Production'].asnumeric()  #.asnumeric()是回归模型，.asfactor()是分类模型
predictors = hf.drop('Hydrogen Production').columns   #predictors作为输入特征，只取列名
response = 'Hydrogen Production'     #response作为输出特征，只取列名

In [ ]:
perf = best_model.model_performance(hf)
print(perf)

In [ ]:
r2 = perf.r2()
print("R2 on the training set:", r2)

In [ ]:
best_model = aml.leader
# 在测试集上进行预测
predictions = best_model.predict(valid_hf)
# 评估模型性能
perf = best_model.model_performance(valid_hf)
print(perf)

In [ ]:
# ra_plot = best_model.residual_analysis_plot(hf)

In [ ]:
# va_plot = aml.varimp_heatmap()
 
# # 或者如果需要一些模型的子集，就可以使用积分排行榜的一部分，
# # 例如，使用MAE作为排序指标
# va_plot = h2o.varimp_heatmap(aml.leaderboard.sort("rmse").head(10))

In [ ]:
# exa = aml.explain(hf)

In [ ]:
# lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
# lb

In [ ]:
# aml.leader.explain(hf)